In [89]:
import itertools #iteration tools
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # nice plots
import matplotlib.pyplot as plt # plots
from ipy_table import *

#preprocessing
from sklearn.preprocessing import LabelEncoder
# machine learning
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# Import and suppress warnings
import warnings
warnings.filterwarnings('ignore')

### Get train Data

In [90]:
train_df = pd.read_csv(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Titanic/train.csv')
test_df = pd.read_csv(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Titanic/test.csv')
both_df = [train_df, test_df]
train_df.head()
super_table=[]
super_table.append(['name', 'reg_function', 'trainset_acc', 'testset_acc', 'r2score', 'conf_matrx'])
df_load = pd.read_csv(r'C:/Users/ernest.chocholowski/Desktop/GIT/Titanic/table_eCh.csv')
for row in df_load.values.tolist():
    super_table.append(row)

In [91]:
#labelling
var_mod = ['Name', 'Sex', 'Ticket','Cabin', 'Embarked']
le = LabelEncoder()
for i in var_mod:
    mask = ~train_df[i].isnull()
    train_df[i][mask] = le.fit_transform(train_df[i][mask])
    mask2 = ~test_df[i].isnull()
    test_df[i][mask2] = le.fit_transform(test_df[i][mask2])
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [92]:
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

In [93]:
#perform imputation!
#imputation of age with imputer
from sklearn.preprocessing import Imputer
imp=Imputer(missing_values="NaN", strategy="mean" )
train_df["Age"]=imp.fit_transform(train_df[["Age"]]).ravel()
test_df["Age"]=imp.fit_transform(test_df[["Age"]]).ravel()

#impute embarked as the most frequent one
freq_port = train_df.Embarked.dropna().mode()[0]
train_df['Embarked'] = train_df['Embarked'].fillna(freq_port)

#impute missing Fare value
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

In [94]:
train_df.info()
print('_'*50)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Embarked       891 non-null int64
dtypes: float64(2), int64(6), object(3)
memory usage: 76.6+ KB
__________________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418

In [95]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,108,1,22.0,1,0,523,7.2500,2
1,2,1,1,190,0,38.0,1,0,596,71.2833,0
2,3,1,3,353,0,26.0,0,0,669,7.9250,2
3,4,1,1,272,0,35.0,1,0,49,53.1000,2
4,5,0,3,15,1,35.0,0,0,472,8.0500,2


In [96]:
def get_regr_function(logreg, y_selection, x_selection):
    coef = logreg.coef_[0]
    intercept = "{:.2f}".format(logreg.intercept_[0])
    output = y_selection + ' = ' + str(intercept) + ' + '
    for coeff, feature in zip(coef, x_selection):
        coeff_str = "{:.2f}".format(coeff)
        output += coeff_str + "*" + feature + " + "
    return output[:-3]

In [97]:
def write_date():
    from time import gmtime, strftime
    return strftime("%Y-%m-%d %H %M %S", gmtime())

In [98]:
def list2str(datalist):
    return " ".join([(str(var)+',') for var in datalist])[:-1]

In [99]:
def R2_score(logreg, X, y, sample_weight=None):
    from sklearn.metrics import r2_score
    return r2_score(y, logreg.predict(X), sample_weight=sample_weight)

In [100]:
def confusion_matrix(model, X, y):
    from sklearn.metrics import confusion_matrix
    return confusion_matrix(y, model.predict(X))

In [101]:
def logist_reg_test (train_df, test_df, x_selection, submission_name = None, x_test_drop = "PassengerId"
                     , y_selection = 'Survived'):
    to_drop = [category for category in train_df.columns.values if category not in x_selection]
    # learning set
    X_train = train_df.drop(to_drop, axis=1)
    # answers for learning set
    Y_train = train_df[y_selection]
    # testing set
    to_drop.remove(y_selection)
    to_drop.append(x_test_drop)
    X_test  = test_df.drop(to_drop, axis=1).copy()
    # Logistic Regression
    logreg = LogisticRegression()
    logreg.fit(X_train, Y_train)
    
    #Calc parameters
    function_str = get_regr_function(logreg, y_selection, x_selection)
    trainset_acc = round(logreg.score(X_train, Y_train) * 100, 2)
    r2_score = R2_score(logreg, X_train, Y_train)

    conf_matrix = confusion_matrix(logreg,X_train,Y_train)
    conf_matrix = 'TN: '+str(conf_matrix[0][0])+', FP: '+str(conf_matrix[0][1])+ \
                      ', FN: '+str(conf_matrix[1][0])+', TP: '+str(conf_matrix[1][1])
    #prints
    print('Regression function:\n', function_str)
    print('Accuracy on train set:', trainset_acc)
    print("R2 score:", r2_score)
    print("Confusion matrix:\n", conf_matrix)
    
    #make predictions
    Y_pred = logreg.predict(X_test)
    #export predictions
    submission = pd.DataFrame({
        "PassengerId": test_df[x_test_drop],
        "Survived": Y_pred
    })
    if submission_name == None:
        submission_name = "regression"+write_date()
    submission.to_csv(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Titanic/output/'+submission_name+'.csv', index=False)
    
    return [submission_name, function_str, str(trainset_acc), r2_score, conf_matrix]

In [30]:
x_test_drop = "PassengerId"
y_selection = 'Survived'
x_selection = train_df.columns.values
x_selection = np.delete(x_selection, [0,1])

In [31]:
output = logist_reg_test(train_df, test_df, x_selection, submission_name='logisticReg_basic_imputation')

Regression function:
 Survived = 4.04 + -0.72*Pclass + -0.00*Name + -2.48*Sex + -0.03*Age + -0.29*SibSp + -0.09*Parch + -0.00*Ticket + 0.01*Fare + -0.15*Embarked
Accuracy on train set: 80.25
R2 score: 0.164797238999
Confusion matrix:
 TN: 479, FP: 70, FN: 106, TP: 236


In [32]:
testset_acc = str('0.75120')
output.insert(3, testset_acc)
super_table.append(output)

In [33]:
def get_regr_fun_lin(logreg, y_selection, x_selection):
    coef = logreg.coef_
    intercept = "{:.2f}".format(logreg.intercept_)
    output = y_selection + ' = ' + str(intercept) + ' + '
    for coeff, feature in zip(coef, x_selection):
        coeff_str = "{:.2f}".format(coeff)
        output += coeff_str + "*" + feature + " + "
    return output[:-3]

In [102]:
def linear_reg_test (train_df, test_df, x_selection, submission_name = None, x_test_drop = "PassengerId"
                     , y_selection = 'Survived'):
    to_drop = [category for category in train_df.columns.values if category not in x_selection]
    # learning set
    X_train = train_df.drop(to_drop, axis=1)
    # answers for learning set
    Y_train = train_df[y_selection]
    # testing set
    to_drop.remove(y_selection)
    to_drop.append(x_test_drop)
    X_test  = test_df.drop(to_drop, axis=1).copy()
    # Logistic Regression
    linreg = LinearRegression()
    linreg.fit(X_train, Y_train)
    #Calc parameters
    function_str = get_regr_fun_lin(linreg, y_selection, x_selection)
    trainset_acc = round(linreg.score(X_train, Y_train) * 100, 2)
    r2_score = R2_score(linreg, X_train, Y_train)

    conf_matrix = "NA"
    
    #prints
    print('Regression function:\n', function_str)
    print('Accuracy on train set:', trainset_acc)
    print("R2 score:", r2_score)
    print("Confusion matrix:\n", conf_matrix)
    
    #make predictions
    Y_pred = linreg.predict(X_test)
    #export predictions
    submission = pd.DataFrame({
        "PassengerId": test_df[x_test_drop],
        "Survived": Y_pred
    })
    if submission_name == None:
        submission_name = "regression"+write_date()
    submission.to_csv(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Titanic/output/'+submission_name+'.csv', 
                      index=False)
    
    return [submission_name, function_str, str(trainset_acc), r2_score, conf_matrix]

In [35]:
x_test_drop = "PassengerId"
y_selection = 'Survived'
x_selection = train_df.columns.values
x_selection = np.delete(x_selection, [0,1])

In [36]:
output = linear_reg_test(train_df, test_df, x_selection, submission_name='linearReg_basic_imputation')

Regression function:
 Survived = 1.37 + -0.15*Pclass + -0.00*Name + -0.51*Sex + -0.01*Age + -0.04*SibSp + -0.02*Parch + -0.00*Ticket + 0.00*Fare + -0.04*Embarked
Accuracy on train set: 40.08
R2 score: 0.400771185732
Confusion matrix:
 NA


In [37]:
testset_acc = str('failed_evl')
output.insert(3, testset_acc)
super_table.append(output)

In [103]:
def other_tests (model, train_df, test_df, x_selection, submission_name = None, x_test_drop = "PassengerId"
                     , y_selection = 'Survived'):
    to_drop = [category for category in train_df.columns.values if category not in x_selection]
    # learning set
    X_train = train_df.drop(to_drop, axis=1)
    # answers for learning set
    Y_train = train_df[y_selection]
    # testing set
    to_drop.remove(y_selection)
    to_drop.append(x_test_drop)
    X_test  = test_df.drop(to_drop, axis=1).copy()
    # Logistic Regression
    model.fit(X_train, Y_train)
    #Calc parameters
    trainset_acc = round(model.score(X_train, Y_train) * 100, 2)
    r2_score = R2_score(model, X_train, Y_train)
    function_str = 'NA'
    conf_matrix = confusion_matrix(model, X_train, Y_train)
    conf_matrix = 'TN: '+str(conf_matrix[0][0])+', FP: '+str(conf_matrix[0][1])+ \
                      ', FN: '+str(conf_matrix[1][0])+', TP: '+str(conf_matrix[1][1])
    
    #prints
    print('Regression function:\n', function_str)
    print('Accuracy on train set:', trainset_acc)
    print("R2 score:", r2_score)
    print("Confusion matrix:\n", conf_matrix)
    
    #make predictions
    Y_pred = model.predict(X_test)
    #export predictions
    submission = pd.DataFrame({
        "PassengerId": test_df[x_test_drop],
        "Survived": Y_pred
    })
    if submission_name == None:
        submission_name = "regression"+write_date()
    submission.to_csv(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Titanic/output/'+submission_name+'.csv', 
                      index=False)
    
    return [submission_name, function_str, str(trainset_acc), r2_score, conf_matrix]

In [104]:
x_test_drop = "PassengerId"
y_selection = 'Survived'
x_selection = train_df.columns.values
x_selection = np.delete(x_selection, [0,1])

In [105]:
# model = SVC()
# model = LinearSVC()
# model = KNeighborsClassifier(n_neighbors = 3)
# model = GaussianNB()
# model = Perceptron()
# model = SGDClassifier()
# model = DecisionTreeClassifier()
# model = RandomForestClassifier(n_estimators=100)

In [106]:
model = RandomForestClassifier(n_estimators=100)
output = other_tests(model, train_df, test_df, x_selection, submission_name='randomForest_basic_imputation')

Regression function:
 NA
Accuracy on train set: 100.0
R2 score: 1.0
Confusion matrix:
 TN: 549, FP: 0, FN: 0, TP: 342


In [107]:
testset_acc = str('75.598')
output.insert(3, testset_acc)

In [66]:
super_table.append(output)

In [80]:
super_table

[['name',
  'reg_function',
  'trainset_acc',
  'testset_acc',
  'r2score',
  'conf_matrx'],
 ['failed_evl',
  0.1696324007807416,
  'TN: 369',
  ' FP: 55',
  ' FN: 88',
  ' TP: 202""'],
 ['failed_evl', 0.4025252345231439, nan, nan, nan, nan],
 ['failed_evl',
  0.9941932335718932,
  'TN: 424',
  ' FP: 0',
  ' FN: 1',
  ' TP: 289""'],
 ['failed_evl',
  0.2741541964866623,
  'TN: 356',
  ' FP: 68',
  ' FN: 57',
  ' TP: 233""'],
 ['failed_evl',
  0.11156473649967458,
  'TN: 352',
  ' FP: 72',
  ' FN: 81',
  ' TP: 209""'],
 ['failed_evl',
  -0.5562134027325961,
  'TN: 417',
  ' FP: 7',
  ' FN: 261',
  ' TP: 29""'],
 ['failed_evl',
  -1.0962426805465193,
  'TN: 94',
  ' FP: 330',
  ' FN: 31',
  ' TP: 259""'],
 ['failed_evl',
  -0.422657774886142,
  'TN: 330',
  ' FP: 94',
  ' FN: 151',
  ' TP: 139""'],
 ['failed_evl', 1.0, 'TN: 424', ' FP: 0', ' FN: 0', ' TP: 290""'],
 ['failed_evl', 1.0, 'TN: 424', ' FP: 0', ' FN: 0', ' TP: 290""'],
 ['75.120',
  0.16479723899913698,
  'TN: 479',
  ' FP: 7

In [87]:
make_table(super_table)
apply_theme('basic')

name,reg_function,trainset_acc,testset_acc,r2score,conf_matrx
failed_evl,0.1696324007807416,TN: 369,FP: 55,FN: 88,"TP: 202"""""
failed_evl,0.4025252345231439,nan,nan,nan,nan
failed_evl,0.9941932335718932,TN: 424,FP: 0,FN: 1,"TP: 289"""""
failed_evl,0.2741541964866623,TN: 356,FP: 68,FN: 57,"TP: 233"""""
failed_evl,0.11156473649967458,TN: 352,FP: 72,FN: 81,"TP: 209"""""
failed_evl,-0.5562134027325961,TN: 417,FP: 7,FN: 261,"TP: 29"""""
failed_evl,-1.0962426805465193,TN: 94,FP: 330,FN: 31,"TP: 259"""""
failed_evl,-0.422657774886142,TN: 330,FP: 94,FN: 151,"TP: 139"""""
failed_evl,1.0,TN: 424,FP: 0,FN: 0,"TP: 290"""""
failed_evl,1.0,TN: 424,FP: 0,FN: 0,"TP: 290"""""


In [68]:
df = pd.DataFrame(super_table[1:], columns=super_table[0])
df.to_csv(r'C:/Users/ernest.chocholowski/Desktop/GIT/Titanic/table_eCh.csv', 
                      index=False)